# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32), 32768 total points
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp="hgh/lda/si-q4")
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.118176e+02     1.547876e+00
 * time: 0.5031578540802002
     1     1.033366e+01     8.657757e-01
 * time: 1.3459548950195312
     2    -1.255654e+01     9.957434e-01
 * time: 1.439823865890503
     3    -3.435975e+01     8.208074e-01
 * time: 1.5704998970031738
     4    -4.799460e+01     5.921460e-01
 * time: 1.6942708492279053
     5    -5.731836e+01     1.816067e-01
 * time: 1.835277795791626
     6    -6.012105e+01     7.707272e-02
 * time: 1.9190959930419922
     7    -6.101761e+01     4.467758e-02
 * time: 2.000734806060791
     8    -6.134452e+01     5.686276e-02
 * time: 2.0818698406219482
     9    -6.163873e+01     2.943039e-02
 * time: 2.166672945022583
    10    -6.184190e+01     2.761128e-02
 * time: 2.3737568855285645
    11    -6.200332e+01     2.531897e-02
 * time: 2.4512338638305664
    12    -6.207750e+01     1.844845e-02
 * time: 2.5293898582458496
    13    -6.214498e+01     1.408596e-02
 * time: 2.60930490493774

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671069
    AtomicLocal         -18.8557696
    AtomicNonlocal      14.8522660
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485376 
    Xc                  -19.3336822

    total               -62.261666461318